In [1]:
import pandas as pd
import os
import itertools
from tqdm.notebook import tqdm
import requests
import json
from datetime import datetime

from dbnomics import fetch_series


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
countries = ['usa', 'eu', 'india', 'ch', 'uk', 'kz', 'jp', 'sk', 'kg', 'turkey'] # 'bel' - не нужна пока что

In [7]:
# clean previous data
for i in countries:
    fname = f'data_processed/{i}_full.csv'
    if os.path.exists(fname):
        os.remove(fname)
if os.path.exists('data_processed/data.csv'):
    os.remove('data_processed/data.csv')

In [3]:
current_month = '05'
current_year = '2025'

In [4]:
url = f'https://sdmx.oecd.org/public/rest/data/OECD.SDD.STES,DSD_KEI@DF_KEI,4.0/JPN+EA19+GBR.M.CC.USD+XDC_USD._Z+_T+BTE..?startPeriod=2002-01&&endPeriod={current_year}-{current_month}&format=jsondata'

In [5]:
url = f'https://sdmx.oecd.org/public/rest/data/OECD.SDD.STES,DSD_STES@DF_FINMARK,4.0/GBR+EA19+JPN.M.CC+IRLT.XDC_USD.....?startPeriod=2002-02&endPeriod={current_year}-{current_month}&format=jsondata'

In [6]:
url 

'https://sdmx.oecd.org/public/rest/data/OECD.SDD.STES,DSD_STES@DF_FINMARK,4.0/GBR+EA19+JPN.M.CC+IRLT.XDC_USD.....?startPeriod=2002-02&endPeriod=2025-05&format=jsondata'

In [7]:
response = requests.get(url)
data = response.json()

In [13]:
#data["data"]["dataSets"][0]["series"]

In [4]:
#data['data']['dataSets'][0]['series']['1:0:0:0:0:0:0:0:0']['observations']

In [5]:
#data['data']['structures'][0]['dimensions']['observation'][0]['values']


In [8]:
# для получения столбцов с курсами валют к доллару, первая дата - 2002-01-01
# нужно заранее обрезать данные нашего общего файла до 2002-01-01, чтобы не было "непереведенных значений"
#response = requests.get(f'https://stats.oecd.org/SDMX-JSON/data/MEI_FIN/CCUS.GBR+EA19+JPN.M/all?startTime=2002-01&endTime={current_year}-{current_month}&detail=DataOnly')
#data = response.json()

yuan = data['dataSets'][0]['series']['0:0:0']['observations']
euro = data['dataSets'][0]['series']['0:2:0']['observations']
funt = data['dataSets'][0]['series']['0:1:0']['observations']
date = data['structure']['dimensions']['observation'][0]['values']

for i in date:
    i.pop('name')

kurs_yuan = pd.DataFrame(date)
kurs_yuan['value'] = yuan.items()
kurs_yuan['value'] = kurs_yuan['value'].apply(lambda x: x[1][0])
kurs_yuan['id'] = kurs_yuan['id'].apply(lambda x: x + '-01')
kurs_yuan = kurs_yuan.rename(columns={'id' : 'PERIOD'})

kurs_eur = pd.DataFrame(date)
kurs_eur['value'] = euro.items()
kurs_eur['value'] = kurs_eur['value'].apply(lambda x: x[1][0])
kurs_eur['id'] = kurs_eur['id'].apply(lambda x: x + '-01')
kurs_eur = kurs_eur.rename(columns={'id' : 'PERIOD'})

kurs_funt = pd.DataFrame(date)
kurs_funt['value'] = funt.items()
kurs_funt['value'] = kurs_funt['value'].apply(lambda x: x[1][0])
kurs_funt['id'] = kurs_funt['id'].apply(lambda x: x + '-01')
kurs_funt = kurs_funt.rename(columns={'id' : 'PERIOD'})

KeyError: 'dataSets'

In [9]:
kurs_funt = fetch_series("IMF/IFS/M.GB.ENDE_XDC_USD_RATE")
kurs_funt = kurs_funt.query("period >= '2002'")
kurs_funt  = kurs_funt[["period", "value"]]
kurs_funt = kurs_funt.rename(columns={'period' : 'PERIOD'})
kurs_funt

,PERIOD,value
540,2002-01-01,0.707664
541,2002-02-01,0.705816
542,2002-03-01,0.702642
543,2002-04-01,0.686436
544,2002-05-01,0.681803
...,...,...
817,2025-02-01,0.793934
818,2025-03-01,0.772827
819,2025-04-01,0.748671
820,2025-05-01,0.742280


In [10]:
kurs_yuan = fetch_series("IMF/IFS/M.JP.ENDE_XDC_USD_RATE")
kurs_yuan = kurs_yuan.query("period >= '2002'")
kurs_yuan  = kurs_yuan[["period", "value"]]
kurs_yuan = kurs_yuan.rename(columns={'period' : 'PERIOD'})
kurs_yuan

,PERIOD,value
540,2002-01-01,132.90
541,2002-02-01,133.90
542,2002-03-01,133.20
543,2002-04-01,128.00
544,2002-05-01,124.40
...,...,...
817,2025-02-01,149.62
818,2025-03-01,149.48
819,2025-04-01,142.52
820,2025-05-01,143.88


In [11]:
kurs_eur = fetch_series("IMF/IFS/M.U2.ENDE_XDC_USD_RATE")
kurs_eur = kurs_eur.query("period >= '2002'")
kurs_eur  = kurs_eur[["period", "value"]]
kurs_eur = kurs_eur.rename(columns={'period' : 'PERIOD'})
kurs_eur

,PERIOD,value
36,2002-01-01,1.157809
37,2002-02-01,1.155936
38,2002-03-01,1.146263
39,2002-04-01,1.110124
40,2002-05-01,1.065303
...,...,...
313,2025-02-01,0.960523
314,2025-03-01,0.924642
315,2025-04-01,0.879275
316,2025-05-01,0.881912


In [28]:
def refresh_ind(YEAR, MONTH): 
    print(f"Processing {YEAR}-{MONTH}")
    df = pd.read_csv(f'INDIA/full{YEAR}{MONTH}.csv')
    
    cols_to_drop = {'Unnamed: 0', 'S.No.', 'Commodity', 'Unit'}
    existing_cols = set(df.columns)
    cols_to_drop = cols_to_drop.intersection(existing_cols)
    
    if cols_to_drop:
        df = df.drop(columns=cols_to_drop)
        
    df = df.rename(columns={'HSCode' : 'TNVED', 'USD' : 'STOIM', 'Quantity' : 'NETTO', 'Flow' : 'NAPR'})
    df['STRANA'] = 'IN'
    df['KOL'] = pd.NA
    df['EDIZM'] = pd.NA
    df['TNVED'] = df['TNVED'].astype(str).apply(lambda x: f'{str(x).zfill(8)}')
    df['TNVED2'] = df['TNVED'].apply(lambda x: x[:2]).apply(lambda x: f'{str(x).zfill(2)}')
    df['TNVED4'] = df['TNVED'].apply(lambda x: x[:4]).apply(lambda x: f'{str(x).zfill(4)}')
    df['TNVED6'] = df['TNVED'].apply(lambda x: x[:6]).apply(lambda x: f'{str(x).zfill(6)}')
   
    df['PERIOD'] = str(YEAR) + '-' + str(MONTH) + '-01'
    df['STOIM'] = df['STOIM'].astype(str).str.replace(',','').astype(float)*10**6
    df['NETTO'] = df['NETTO'].astype(str).str.replace(',','').astype(float)    
    df.loc[(df['NAPR'] == 'Im'),'NAPR'] = 'ЭК'
    df.loc[(df['NAPR'] == 'Ex'),'NAPR'] = 'ИМ'
    df = df.query('STOIM.notna() or NETTO.notna()')
    df = df.reindex(columns=['NAPR', 'PERIOD', 'STRANA', 'TNVED', 'EDIZM', 'STOIM', 'NETTO', 'KOL', 'TNVED4', 'TNVED6', 'TNVED2'])
    file = 'data_processed/india_full.csv'
    if not os.path.exists(file):
        df.to_csv(file, header='column_names', index=False, encoding='utf-8-sig')
    else: # else it exists so append without writing the header
        df.to_csv(file, mode='a', header=False, index=False, encoding='utf-8-sig')

In [51]:
def refresh_eu(YEAR, MONTH):
    if int(YEAR)<2012:
        return
    df = pd.read_csv(f'EU/full{YEAR}{MONTH}.csv')
    df = df.rename(columns=(lambda x: x.upper()))
    df = df.drop(columns={'UNNAMED: 0', 'DECLARANT', 'PARTNER', 'TRADE_TYPE', 'PRODUCT_SITC',\
         'PRODUCT_CPA2002', 'PRODUCT_CPA2008', 'PRODUCT_CPA2_1', 'PRODUCT_BEC', 'PRODUCT_SECTION',\
              'PARTNER_ISO', 'STAT_REGIME'})
    df = df.rename(columns={'PRODUCT_NC' : 'TNVED',\
                'VALUE_IN_EUROS' : 'STOIM', 'QUANTITY_IN_KG' : 'NETTO', 'SUPP_UNIT' : 'EDIZM',\
                        'SUP_QUANTITY' : 'KOL', 'DECLARANT_ISO' : 'STRANA', 'FLOW' : 'NAPR'})
    df.query('TNVED != "TOTAL"', inplace=True)

    df.loc[(df['EDIZM'].isnull()),'KOL'] = pd.NA
    df.loc[(df['KOL'].isnull()),'EDIZM'] = '-'

    df.loc[(df['NAPR'] == 1),'NAPR'] = 'ИМ'
    df.loc[(df['NAPR'] == 2),'NAPR'] = 'ЭК'

    df['TNVED'] = df['TNVED'].astype(str).apply(lambda x: f'{str(x).zfill(8)}')
    df['TNVED2'] = df['TNVED'].apply(lambda x: x[:2]).apply(lambda x: f'{str(x).zfill(2)}')
    df['TNVED4'] = df['TNVED'].apply(lambda x: x[:4]).apply(lambda x: f'{str(x).zfill(4)}')
    df['TNVED6'] = df['TNVED'].apply(lambda x: x[:6]).apply(lambda x: f'{str(x).zfill(6)}')
    df['STOIM'] = df['STOIM'].astype(str).str.replace(',','').astype(float)
    df['NETTO'] = df['NETTO'].astype(str).str.replace(',','').astype(float)    

    df['y'] = df['PERIOD'].astype(str).apply(lambda x: x[:4])
    df['m'] = df['PERIOD'].astype(str).apply(lambda x: x[4:])
    df['PERIOD'] = df['y'] + '-' + df['m'] + '-01'
    df = df.drop(columns={'y','m'})
    df = df.query('STOIM.notna() or NETTO.notna()')
    df = df.reindex(columns=['NAPR', 'PERIOD', 'STRANA', 'TNVED', 'EDIZM', 'STOIM', 'NETTO', 'KOL', 'TNVED4', 'TNVED6', 'TNVED2'])
    df = df.query('STRANA != "GB"') # remove UK data because we have it separately

    # преобразование евро в доллары
    df["PERIOD"] = pd.to_datetime(df["PERIOD"])
    df = df.merge(kurs_eur, on='PERIOD')
    df['STOIM'] = df['STOIM'] / df['value']
    df = df.drop(columns={'value'})

    file = 'data_processed/eu_full.csv'
    if not os.path.exists(file):
        df.to_csv(file, header='column_names', index=False, encoding='utf-8-sig')
    else: # else it exists so append without writing the header
        df.to_csv(file, mode='a', header=False, index=False, encoding='utf-8-sig')

In [16]:
def refresh_usa(YEAR, MONTH):
    df = pd.read_csv(f'USA/data{YEAR}{MONTH}.csv')
    df = df.drop(columns={'CTY_NAME'})
    df['STOIM'] = df['STOIM'].astype(str).str.replace(',','').astype(float)
    df['NETTO'] = df['NETTO'].astype(str).str.replace(',','').astype(float)   
    df['STRANA'] = 'US'
    df['PERIOD'] = df['PERIOD'] + '-01'
    df = df.reindex(columns=['NAPR', 'PERIOD', 'STRANA', 'TNVED', 'EDIZM', 'STOIM', 'NETTO', 'KOL', 'TNVED4', 'TNVED6', 'TNVED2'])

    file = 'data_processed/usa_full.csv'
    if not os.path.exists(file):
        df.to_csv(file, header='column_names', index=False, encoding='utf-8-sig')
    else: # else it exists so append without writing the header
        df.to_csv(file, mode='a', header=False, index=False, encoding='utf-8-sig')

In [10]:
def refresh_ch(YEAR, MONTH, TYPE):
    if TYPE == 'ЭК':
        df = pd.read_csv(f'CHINA/EXPORT/data{YEAR}{MONTH}.csv')
    elif TYPE == 'ИМ':
        df = pd.read_csv(f'CHINA/IMPORT/data{YEAR}{MONTH}.csv')
    df['PERIOD'] = df['PERIOD'] + '-01'
    df = df.reindex(columns=['NAPR', 'PERIOD', 'STRANA', 'TNVED', 'EDIZM', 'STOIM', 'NETTO', 'KOL', 'TNVED4', 'TNVED6', 'TNVED2'])
    df['STRANA'] = 'CN'
    df['TNVED'] = df['TNVED'].apply(lambda x: f'{str(x).zfill(8)}')
    df['TNVED2'] = df['TNVED2'].apply(lambda x: f'{str(x).zfill(2)}')
    df['TNVED4'] = df['TNVED4'].apply(lambda x: f'{str(x).zfill(4)}')
    df['TNVED6'] = df['TNVED6'].apply(lambda x: f'{str(x).zfill(6)}')

    file = 'data_processed/ch_full.csv'
    if not os.path.exists(file):
        df.to_csv(file, header='column_names', index=False, encoding='utf-8-sig')
    else: # else it exists so append without writing the header
        df.to_csv(file, mode='a', header=False, index=False, encoding='utf-8-sig')

In [42]:
def refresh_uk(YEAR, MONTH, TYPE):
    if int(YEAR)<2002:
        return
    df = pd.DataFrame()
    if TYPE == 'ЭК':
        test = pd.read_csv(f'UK/IMPORT/BDSimp{str(YEAR)[2:]}{MONTH}.txt', header=None)
    elif TYPE == 'ИМ':
        test = pd.read_csv(f'UK/EXPORT/BDSexp{str(YEAR)[2:]}{MONTH}.txt', header=None)
    df['PERIOD'] = test[0].apply(lambda x: x[:4] + '-' + x[4:6] + '-01')
    df['TNVED'] = test[0].apply(lambda x: x[13:21])
    df['STRANA'] = test[0].apply(lambda x: x[29:31])
    df['STOIM'] = test[0].apply(lambda x: x[44:56]).astype(float)
    df['NETTO'] = test[0].apply(lambda x: x[56:68])
    df.query('NETTO != "           ."', inplace = True)
    df['NETTO'] = df['NETTO'].astype(float)
    df['KOL'] = test[0].apply(lambda x: x[68:80])
    df.query('KOL != "           ."', inplace = True)
    df['KOL'] = df['KOL'].astype(float)
    df['EDIZM'] = '-'
    df['TNVED_LAST'] = df['TNVED'].apply(lambda x: x[-1])
    df.query('TNVED_LAST != "-"', inplace = True)
    df = df.drop(columns={'TNVED_LAST'})
    df['TNVED2'] = df['TNVED'].apply(lambda x: x[:2]).apply(lambda x: f'{str(x).zfill(2)}')
    df['TNVED4'] = df['TNVED'].apply(lambda x: x[:4]).apply(lambda x: f'{str(x).zfill(4)}')
    df['TNVED6'] = df['TNVED'].apply(lambda x: x[:6]).apply(lambda x: f'{str(x).zfill(6)}')

    df = df.reindex(columns=['NAPR', 'PERIOD', 'STRANA', 'TNVED', 'EDIZM', 'STOIM', 'NETTO', 'KOL', 'TNVED4', 'TNVED6', 'TNVED2'])
    df.query('STRANA == "RU"', inplace = True)
    df['STRANA'] = 'GB'
    df['NAPR'] = TYPE
    
    # преобразование фунтов в доллары
    df["PERIOD"] = pd.to_datetime(df["PERIOD"])
    df = df.merge(kurs_funt, on='PERIOD')
    df['STOIM'] = df['STOIM'] / df['value']
    df = df.drop(columns={'value'})

    file = 'data_processed/uk_full.csv'
    if not os.path.exists(file):
        df.to_csv(file, header='column_names', index=False, encoding='utf-8-sig')
    else: # else it exists so append without writing the header
        df.to_csv(file, mode='a', header=False, index=False, encoding='utf-8-sig')

In [12]:
def refresh_bel(YEAR, MONTH):
    df = pd.read_csv(f'BELARUS/data{YEAR}{MONTH}.csv')

    file = 'data_processed/bel_full.csv'
    if not os.path.exists(file):
        df.to_csv(file, header='column_names', index=False, encoding='utf-8-sig')
    else: # else it exists so append without writing the header
        df.to_csv(file, mode='a', header=False, index=False, encoding='utf-8-sig')

In [13]:
def refresh_kz(YEAR, MONTH):

    filename = f'KZ/data{YEAR}{MONTH}.xls'
    # проверка, на каком файле может случиться ошибка, файл data202203.xls был 
    # изначально поврежденный, я скачивал его пару раз для проверки, поэтому без него
    df_st = pd.read_excel(filename)
    df_st = df_st.rename(columns={df_st.columns[0] : 'TNVED', 'Unnamed: 1' : 'STRANA'})
    df_st['STRANA'] = df_st['STRANA'].str.lower()
    df_st['TNVED'] = df_st['TNVED'].fillna(method='ffill')
    
    df_st = df_st.query('STRANA == "россия"') 
    df_imp = df_st[['TNVED', 'STRANA', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']]

    df_imp = df_imp.rename(columns={'Unnamed: 2' : 'EDIZM', 'Unnamed: 3' : 'NETTO',\
                                'Unnamed: 4' : 'KOL', 'Unnamed: 5' : 'STOIM'})
    df_imp['NAPR'] = 'ИМ'
    df_imp['STRANA'] = 'KZ'

    df_exp = df_st[['TNVED', 'STRANA', 'Unnamed: 2', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8']]

    df_exp = df_exp.rename(columns={'Unnamed: 2' : 'EDIZM', 'Unnamed: 6' : 'NETTO',\
                                'Unnamed: 7' : 'KOL', 'Unnamed: 8' : 'STOIM'})
    df_exp['NAPR'] = 'ЭК'
    df_exp['STRANA'] = 'KZ'

    df = pd.concat([df_exp, df_imp], ignore_index=True)
    df['TNVED2'] = df['TNVED'].apply(lambda x: x[:2]).apply(lambda x: f'{str(x).zfill(2)}')
    df['TNVED4'] = df['TNVED'].apply(lambda x: x[:4]).apply(lambda x: f'{str(x).zfill(4)}')
    df['TNVED6'] = df['TNVED'].apply(lambda x: x[:6]).apply(lambda x: f'{str(x).zfill(6)}')
    df['STOIM'] = df['STOIM'] * 1000
    df['NETTO'] = df['NETTO'] * 1000

    df = df.query('STOIM < 100000000')


    df['PERIOD'] = filename[7:11] + '-' + filename[11:13] + '-01'

    df = df.reindex(columns=['NAPR', 'PERIOD', 'STRANA', 'TNVED', 'EDIZM', 'STOIM', 'NETTO', 'KOL', 'TNVED4', 'TNVED6', 'TNVED2'])


    file = 'data_processed/kz_full.csv'
    if not os.path.exists(file):
        df.to_csv(file, header='column_names', index=False, encoding='utf-8-sig')
    else: # else it exists so append without writing the header
        df.to_csv(file, mode='a', header=False, index=False, encoding='utf-8-sig')

In [48]:
def refresh_jp(YEAR):

    filename_exp = f'JAPAN/data{YEAR}_exp.csv'
    filename_imp = f'JAPAN/data{YEAR}_imp.csv'

    df_exp = pd.read_csv(filename_exp, on_bad_lines='skip')
    df_exp['Exp or Imp'] = 'ИМ'
    df_imp = pd.read_csv(filename_imp, on_bad_lines='skip')
    df_imp['Exp or Imp'] = 'ЭК'

    df = pd.concat([df_exp, df_imp], ignore_index=True)
    df = df.query('Country == 224 or Country == "224"')
    df = df.rename(columns={'Exp or Imp' : 'NAPR', 'Year' : 'PERIOD', 'HS' : 'TNVED', 'Country' : 'STRANA'})
    df['STRANA'] = 'JP'
    df.loc[(df['Unit2'] != 'KG'),'Unit1'] = df['Unit2']


    file = 'data_processed/jp_full.csv'
    
    months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    for month in months:

        # в 2014-2016 годах у них есть ошибка - они называют столбцы с апрелем "Apl" вместо "Apr"
        if YEAR in ['2014', '2015', '2016'] and month == 'Apr': 
            df_month = df[['NAPR', 'PERIOD', 'TNVED', 'STRANA', 'Unit1', 'Unit2',  'Quantity1-Apl', 'Quantity2-Apl', 'Value-Apl']]
            df_month.loc[(df_month['Unit2'] != 'KG'),'Quantity1-Apl'] = df_month['Quantity2-Apl']
            df_month.loc[(df_month['Unit2'] != 'KG'),'Quantity2-Apl'] = pd.NA
            df_month = df_month.drop(columns={'Unit2'})
            
            MONTH = '04'

            df_month['PERIOD'] = YEAR + '-' + MONTH + '-01'
            df_month["PERIOD"] = pd.to_datetime(df_month["PERIOD"])
            df_month['TNVED'] = df_month['TNVED'].apply(lambda x: x[1:len(x)-1]).apply(lambda x: f'{str(x).zfill(9)}')
            df_month['TNVED2'] = df_month['TNVED'].apply(lambda x: x[:2]).apply(lambda x: f'{str(x).zfill(2)}')
            df_month['TNVED4'] = df_month['TNVED'].apply(lambda x: x[:4]).apply(lambda x: f'{str(x).zfill(4)}')
            df_month['TNVED6'] = df_month['TNVED'].apply(lambda x: x[:6]).apply(lambda x: f'{str(x).zfill(6)}')
            df_month = df_month.rename(columns={'Unit1' : 'EDIZM', 'Quantity1-Apl' : 'KOL', 'Quantity2-Apl' : 'NETTO', 'Value-Apl' : 'STOIM'})
            df_month = df_month.reindex(columns=['NAPR', 'PERIOD', 'STRANA', 'TNVED', 'EDIZM', 'STOIM', 'NETTO', 'KOL', 'TNVED4', 'TNVED6', 'TNVED2'])
            df_month['STOIM'] = df_month['STOIM'].astype(float) * 1000
            # преобразование юани в доллары
            df_month = df_month.merge(kurs_yuan, on='PERIOD')
            df_month['STOIM'] = df_month['STOIM'] / df_month['value']
            df_month = df_month.drop(columns={'value'})

        else: 
            df_month = df[['NAPR', 'PERIOD', 'TNVED', 'STRANA', 'Unit1', 'Unit2',  f'Quantity1-{month}', f'Quantity2-{month}', f'Value-{month}']]
            df_month.loc[(df_month['Unit2'] != 'KG'),f'Quantity1-{month}'] = df_month[f'Quantity2-{month}']
            df_month.loc[(df_month['Unit2'] != 'KG'),f'Quantity2-{month}'] = pd.NA
            df_month = df_month.drop(columns={'Unit2'})
            if month == 'Jan':
                MONTH = '01'
            if month == 'Feb':
                MONTH = '02'
            if month == 'Mar':
                MONTH = '03'
            if month == 'Apr':
                MONTH = '04'
            if month == 'May':
                MONTH = '05'
            if month == 'Jun':
                MONTH = '06'
            if month == 'Jul':
                MONTH = '07'
            if month == 'Aug':
                MONTH = '08'
            if month == 'Sep':
                MONTH = '09'
            if month == 'Oct':
                MONTH = '10'
            if month == 'Nov':
                MONTH = '11'
            if month == 'Dec':
                MONTH = '12'
            df_month['PERIOD'] = YEAR + '-' + MONTH + '-01'
            df_month["PERIOD"] = pd.to_datetime(df_month["PERIOD"])
            df_month['TNVED'] = df_month['TNVED'].apply(lambda x: x[1:len(x)-1]).apply(lambda x: f'{str(x).zfill(9)}')
            df_month['TNVED2'] = df_month['TNVED'].apply(lambda x: x[:2]).apply(lambda x: f'{str(x).zfill(2)}')
            df_month['TNVED4'] = df_month['TNVED'].apply(lambda x: x[:4]).apply(lambda x: f'{str(x).zfill(4)}')
            df_month['TNVED6'] = df_month['TNVED'].apply(lambda x: x[:6]).apply(lambda x: f'{str(x).zfill(6)}')
            df_month = df_month.rename(columns={'Unit1' : 'EDIZM', f'Quantity1-{month}' : 'KOL', f'Quantity2-{month}' : 'NETTO', f'Value-{month}' : 'STOIM'})
            df_month = df_month.reindex(columns=['NAPR', 'PERIOD', 'STRANA', 'TNVED', 'EDIZM', 'STOIM', 'NETTO', 'KOL', 'TNVED4', 'TNVED6', 'TNVED2'])
            
            
            df_month['STOIM'] = df_month['STOIM'].apply(lambda x: round(float(x),4))

            df_month['STOIM'] = df_month['STOIM'].astype(float) * 1000
            # преобразование юани в доллары
            df_month["PERIOD"] = pd.to_datetime(df_month["PERIOD"])
            df_month = df_month.merge(kurs_yuan, on='PERIOD')
            df_month['STOIM'] = df_month['STOIM'] / df_month['value']
            df_month = df_month.drop(columns={'value'})

        if not os.path.exists(file):
            df_month.to_csv(file, header='column_names', index=False, encoding='utf-8-sig')
        else: # else it exists so append without writing the header
            df_month.to_csv(file, mode='a', header=False, index=False, encoding='utf-8-sig')

In [15]:
def refresh_sk(YEAR, MONTH):
    df = pd.read_csv(f'SK/data{YEAR}{MONTH}.csv')
    
    df['TNVED'] = df['TNVED'].apply(lambda x: f'{str(x).zfill(6)}')
    df['TNVED2'] = df['TNVED'].apply(lambda x: x[:2]).apply(lambda x: f'{str(x).zfill(2)}')
    df['TNVED4'] = df['TNVED'].apply(lambda x: x[:4]).apply(lambda x: f'{str(x).zfill(4)}')
    df['TNVED6'] = df['TNVED'].apply(lambda x: x[:6]).apply(lambda x: f'{str(x).zfill(6)}')

    df['PERIOD'] = YEAR + '-' + MONTH + '-01'
    
    df['NETTO'] = df['NETTO'].apply(lambda x: x.replace(',','')).apply(lambda x: float(x))
    df['STOIM'] = df['STOIM'].apply(lambda x: x.replace(',','')).apply(lambda x: float(x))

    df['NETTO'] = df['NETTO'] * 1000
    df['STOIM'] = df['STOIM'] * 1000

    df['EDIZM'] = pd.NA
    df['KOL'] = pd.NA

    df = df.reindex(columns=['NAPR', 'PERIOD', 'STRANA', 'TNVED', 'EDIZM', 'STOIM', 'NETTO', 'KOL', 'TNVED4', 'TNVED6', 'TNVED2'])

    file = 'data_processed/sk_full.csv'
    if not os.path.exists(file):
        df.to_csv(file, header='column_names', index=False, encoding='utf-8-sig')
    else: # else it exists so append without writing the header
        df.to_csv(file, mode='a', header=False, index=False, encoding='utf-8-sig')

In [42]:
YEAR=2023

In [43]:
df = pd.read_csv(f'TURKEY/data{2022}.csv', header=None, on_bad_lines='skip')
df = df.tail(-1)
df

D:\Anaconda2022\envs\gie\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (0,1,2,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
1,2022,2022,1,75,Russia,03021180,"Fresh or chilled trout ""Salmo trutta, Oncorhyn...","Fresh or chilled trout ""Salmo trutta, Oncorhyn...","Fresh or chilled trout ""Salmo trutta, Oncorhyn...","Fresh or chilled trout ""Salmo trutta, Oncorhyn...","Fresh or chilled trout ""Salmo trutta, Oncorhyn...",KG,20.964,0,0,0,0,83.308,0
2,2022,2022,1,75,Russia,54078300,Woven fabrics of yarn containing predominantly...,Woven fabrics of yarn containing predominantly...,Woven fabrics of yarn containing predominantly...,Woven fabrics of yarn containing predominantly...,Woven fabrics of yarn containing predominantly...,KG/M2,12.763,91.247,91.247,0,0,80.552,0
3,2022,2022,1,75,Russia,83023000,"Base metal mountings, fittings and similar art...","Base metal mountings, fittings and similar art...","Base metal mountings, fittings and similar art...","Base metal mountings, fittings and similar art...","Base metal mountings, fittings and similar art...",KG,57.051,0,0,160,0,435.058,4.293
4,2022,2022,1,75,Russia,90330090,"Parts and accessories for machines, appliances...","Parts and accessories for machines, appliances...","Parts and accessories for machines, appliances...","Parts and accessories for machines, appliances...","Parts and accessories for machines, appliances...",KG,1.025,0,0,6,0,20.268,164
5,2022,2022,1,75,Russia,95059000,"Festival, carnival or other entertainment arti...","Festival, carnival or other entertainment arti...","Festival, carnival or other entertainment arti...","Festival, carnival or other entertainment arti...","Festival, carnival or other entertainment arti...",KG,8,0,0,3,0,214,135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34935,2022,2022,11,75,Russia,54077300,Woven fabrics of yarn containing >= 85% synthe...,Woven fabrics of yarn containing >= 85% synthe...,Woven fabrics of yarn containing >= 85% synthe...,Woven fabrics of yarn containing >= 85% synthe...,Woven fabrics of yarn containing >= 85% synthe...,KG/M2,35.552,354.190,354.190,0,0.0,165.553,0
34936,2022,2022,11,75,Russia,84279000,Works trucks fitted with lifting or handling e...,Works trucks fitted with lifting or handling e...,Works trucks fitted with lifting or handling e...,Works trucks fitted with lifting or handling e...,Works trucks fitted with lifting or handling e...,KG/ADET,160,2,2,0,0.0,502,0
34937,2022,2022,12,75,Russia,29053920,"Butane-1,3-diol","Butane-1,3-diol","Butane-1,3-diol","Butane-1,3-diol","Butane-1,3-diol",KG,8,0,0,0,0.0,51,0
34938,2022,2022,12,75,Russia,54077300,Woven fabrics of yarn containing >= 85% synthe...,Woven fabrics of yarn containing >= 85% synthe...,Woven fabrics of yarn containing >= 85% synthe...,Woven fabrics of yarn containing >= 85% synthe...,Woven fabrics of yarn containing >= 85% synthe...,KG/M2,43.005,375.192,375.192,0,0.0,208.839,0


In [71]:
df2 = pd.read_csv(f'TURKEY/data{2023}.csv', header=None, on_bad_lines='skip')
df2 = df2.tail(-1)
df2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
1,2023,2023,1,75,Russia,07093000,"Fresh or chilled aubergines ""eggplants""","Fresh or chilled aubergines ""eggplants""","Fresh or chilled aubergines ""eggplants""","Fresh or chilled aubergines ""eggplants""","Fresh or chilled aubergines ""eggplants""",KG,81.220,0,0,0,0,103.167,0
2,2023,2023,1,75,Russia,19043000,"Bulgur wheat in the form of worked grains, obt...","Bulgur wheat in the form of worked grains, obt...","Bulgur wheat in the form of worked grains, obt...","Bulgur wheat in the form of worked grains, obt...","Bulgur wheat in the form of worked grains, obt...",KG,1.597.022,0,0,0,0,1.091.593,0
3,2023,2023,1,75,Russia,29362800,"Vitamin E and its derivatives, used primarily ...","Vitamin E and its derivatives, used primarily ...","Vitamin E and its derivatives, used primarily ...","Vitamin E and its derivatives, used primarily ...","Vitamin E and its derivatives, used primarily ...",KG,2,0,0,0,0,113,0
4,2023,2023,1,75,Russia,44089095,"Sheets for veneering, incl. those obtained by ...","Sheets for veneering, incl. those obtained by ...","Sheets for veneering, incl. those obtained by ...","Sheets for veneering, incl. those obtained by ...","Sheets for veneering, incl. those obtained by ...",KG/M3,0,0,0,7.584.790,12.195,0,4.189.876
5,2023,2023,1,75,Russia,55132390,"Woven fabrics containing predominantly, but < ...","Woven fabrics containing predominantly, but < ...","Woven fabrics containing predominantly, but < ...","Woven fabrics containing predominantly, but < ...","Woven fabrics containing predominantly, but < ...",KG/M2,752,2.905,2.905,0,0,13.203,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10046,2023,2023,1,75,Russia,85232915,"Magnetic tapes and magnetic discs, unrecorded,...","Magnetic tapes and magnetic discs, unrecorded,...","Magnetic tapes and magnetic discs, unrecorded,...","Magnetic tapes and magnetic discs, unrecorded,...","Magnetic tapes and magnetic discs, unrecorded,...",KG/ADET,8,1,1,0,0,239,0
10047,2023,2023,2,75,Russia,84272019,"Works trucks, self-propelled, with a lifting h...","Works trucks, self-propelled, with a lifting h...","Works trucks, self-propelled, with a lifting h...","Works trucks, self-propelled, with a lifting h...","Works trucks, self-propelled, with a lifting h...",KG/ADET,7.360,1,1,0,0,69.121,0
10048,2023,2023,3,75,Russia,82082000,"Knives and cutting blades, of base metal, for ...","Knives and cutting blades, of base metal, for ...","Knives and cutting blades, of base metal, for ...","Knives and cutting blades, of base metal, for ...","Knives and cutting blades, of base metal, for ...",KG,600,0,0,0,0,1.590,0
10049,2023,2023,3,75,Russia,84272019,"Works trucks, self-propelled, with a lifting h...","Works trucks, self-propelled, with a lifting h...","Works trucks, self-propelled, with a lifting h...","Works trucks, self-propelled, with a lifting h...","Works trucks, self-propelled, with a lifting h...",KG/ADET,105.570,13,13,0,0,920.237,0


In [16]:
def refresh_turkey(YEAR):
    print(YEAR)
    df = pd.read_csv(f'TURKEY/data{YEAR}.csv', header=None, on_bad_lines='skip')
    df = df.tail(-1)
    
#     df['PERIOD'] = df[0].apply(lambda x: str(x)) + '-' + df[2].apply(lambda x: str(x)).apply(lambda x: f'{str(x).zfill(2)}') + '-01'
    
    df['PERIOD'] = str(YEAR) + '-' + df[2].astype(str).str.zfill(2) + '-01'
    
    #if YEAR == '2022':
    #    df = df.drop(columns={0,1,2,3,6,7,8})
    #else:
    #    df = df.drop(columns={0,1,2,3,6,7,8,9,10,14})
    df = df.drop(columns={0,1,2,3,6,7,8,9,10,14})  
    df[4] = 'TR'
    df['TNVED'] = df[5].astype(str).apply(lambda x: f'{str(x).zfill(8)}')
    df['TNVED2'] = df[5].astype(str).apply(lambda x: x[:2]).apply(lambda x: f'{str(x).zfill(2)}')
    df['TNVED4'] = df[5].astype(str).apply(lambda x: x[:4]).apply(lambda x: f'{str(x).zfill(4)}')
    df['TNVED6'] = df[5].astype(str).apply(lambda x: x[:6]).apply(lambda x: f'{str(x).zfill(6)}')
#     if YEAR == '2022':
#         df['EDIZM'] = df[9].astype(str).apply(lambda x: x[3:] if len(x)>2 else pd.NA)
#         df_imp = df[[4,12,13,15,'PERIOD','TNVED', 'TNVED2', 'TNVED4', 'TNVED6', 'EDIZM']]
#         df_exp = df[[4,10,11,14,'PERIOD','TNVED', 'TNVED2', 'TNVED4', 'TNVED6', 'EDIZM']]
#         df_imp = df_imp.rename(columns={4 : 'STRANA', 12 : 'NETTO', 13 : 'KOL', 15 : 'STOIM', 'Country' : 'STRANA'})
#         df_imp['NAPR'] = 'ЭК'
#         df_exp = df_exp.rename(columns={4 : 'STRANA', 10 : 'NETTO', 11 : 'KOL', 14 : 'STOIM', 'Country' : 'STRANA'})
#         df_exp['NAPR'] = 'ИМ'
#     else:
#         df['EDIZM'] = df[11].astype(str).apply(lambda x: x[3:] if len(x)>2 else pd.NA) 
#         df_imp = df[[4,15,16,18,'PERIOD','TNVED', 'TNVED2', 'TNVED4', 'TNVED6', 'EDIZM']]
#         df_exp = df[[4,12,13,17,'PERIOD','TNVED', 'TNVED2', 'TNVED4', 'TNVED6', 'EDIZM']]
#         df_imp = df_imp.rename(columns={4 : 'STRANA', 15 : 'NETTO', 16 : 'KOL', 18 : 'STOIM', 'Country' : 'STRANA'})
#         df_imp['NAPR'] = 'ЭК'
#         df_exp = df_exp.rename(columns={4 : 'STRANA', 12 : 'NETTO', 13 : 'KOL', 17 : 'STOIM', 'Country' : 'STRANA'})
#         df_exp['NAPR'] = 'ИМ'
        
    df['EDIZM'] = df[11].astype(str).apply(lambda x: x[3:] if len(x)>2 else pd.NA) 
    df_imp = df[[4,15,16,18,'PERIOD','TNVED', 'TNVED2', 'TNVED4', 'TNVED6', 'EDIZM']]
    df_exp = df[[4,12,13,17,'PERIOD','TNVED', 'TNVED2', 'TNVED4', 'TNVED6', 'EDIZM']]
    df_imp = df_imp.rename(columns={4 : 'STRANA', 15 : 'NETTO', 16 : 'KOL', 18 : 'STOIM', 'Country' : 'STRANA'})
    df_imp['NAPR'] = 'ЭК'
    df_exp = df_exp.rename(columns={4 : 'STRANA', 12 : 'NETTO', 13 : 'KOL', 17 : 'STOIM', 'Country' : 'STRANA'})
    df_exp['NAPR'] = 'ИМ'

    df = pd.concat([df_imp, df_exp], ignore_index=True)

    df['NETTO'] = df['NETTO'].astype(str).apply(lambda x: x.replace('.',''))
    df['STOIM'] = df['STOIM'].astype(str).apply(lambda x: x.replace('.',''))
    df['KOL'] = df['KOL'].astype(str).apply(lambda x: x.replace('.',''))

    df = df.reindex(columns=['NAPR', 'PERIOD', 'STRANA', 'TNVED', 'EDIZM', 'STOIM', 'NETTO', 'KOL', 'TNVED4', 'TNVED6', 'TNVED2'])
    df["PERIOD"] = df["PERIOD"].astype(str)
    df = df[~df["PERIOD"].str.contains('-75-01')]
    
    file = 'data_processed/turkey_full.csv'
    if not os.path.exists(file):
        df.to_csv(file, header='column_names', index=False, encoding='utf-8-sig')
    else: # else it exists so append without writing the header
        df.to_csv(file, mode='a', header=False, index=False, encoding='utf-8-sig')

In [17]:
def refresh_KG(YEAR):
    df = pd.read_csv(f'KG/data{YEAR}.csv', on_bad_lines='skip')
    file = 'data_processed/kg_full.csv'
    if not os.path.exists(file):
        df.to_csv(file, header='column_names', index=False, encoding='utf-8-sig')
    else: # else it exists so append without writing the header
        df.to_csv(file, mode='a', header=False, index=False, encoding='utf-8-sig')

In [15]:
to_rebuild_ind = [ (int(x.split('.')[0][4:8]), int(x.split('.')[0][8:10])) for x in os.listdir('INDIA')]
to_rebuild_ind

[(2010, 1),
 (2010, 2),
 (2010, 3),
 (2010, 4),
 (2010, 5),
 (2010, 6),
 (2010, 7),
 (2010, 8),
 (2010, 9),
 (2010, 10),
 (2010, 11),
 (2010, 12),
 (2011, 1),
 (2011, 2),
 (2011, 3),
 (2011, 4),
 (2011, 5),
 (2011, 6),
 (2011, 7),
 (2011, 8),
 (2011, 9),
 (2011, 10),
 (2011, 11),
 (2011, 12),
 (2012, 1),
 (2012, 2),
 (2012, 3),
 (2012, 4),
 (2012, 5),
 (2012, 6),
 (2012, 7),
 (2012, 8),
 (2012, 9),
 (2012, 10),
 (2012, 11),
 (2012, 12),
 (2013, 1),
 (2013, 2),
 (2013, 3),
 (2013, 4),
 (2013, 5),
 (2013, 6),
 (2013, 7),
 (2013, 8),
 (2013, 9),
 (2013, 10),
 (2013, 11),
 (2013, 12),
 (2014, 1),
 (2014, 2),
 (2014, 3),
 (2014, 4),
 (2014, 5),
 (2014, 6),
 (2014, 7),
 (2014, 8),
 (2014, 9),
 (2014, 10),
 (2014, 11),
 (2014, 12),
 (2015, 1),
 (2015, 2),
 (2015, 3),
 (2015, 4),
 (2015, 5),
 (2015, 6),
 (2015, 7),
 (2015, 8),
 (2015, 9),
 (2015, 10),
 (2015, 11),
 (2015, 12),
 (2016, 1),
 (2016, 2),
 (2016, 3),
 (2016, 4),
 (2016, 5),
 (2016, 6),
 (2016, 7),
 (2016, 8),
 (2016, 9),
 (2016, 10

In [12]:
to_rebuild_usa = [ (int(x.split('.')[0][4:8]), int(x.split('.')[0][8:10])) for x in os.listdir('USA')]

In [14]:
to_rebuild_usa = [ (int(x.split('.')[0][4:8]), int(x.split('.')[0][8:10])) for x in os.listdir('USA')]
to_rebuild_usa

[(2014, 1),
 (2014, 2),
 (2014, 3),
 (2014, 4),
 (2014, 5),
 (2014, 6),
 (2014, 7),
 (2014, 8),
 (2014, 9),
 (2014, 10),
 (2014, 11),
 (2014, 12),
 (2015, 1),
 (2015, 2),
 (2015, 3),
 (2015, 4),
 (2015, 5),
 (2015, 6),
 (2015, 7),
 (2015, 8),
 (2015, 9),
 (2015, 10),
 (2015, 11),
 (2015, 12),
 (2016, 1),
 (2016, 2),
 (2016, 3),
 (2016, 4),
 (2016, 5),
 (2016, 6),
 (2016, 7),
 (2016, 8),
 (2016, 9),
 (2016, 10),
 (2016, 11),
 (2016, 12),
 (2017, 1),
 (2017, 2),
 (2017, 3),
 (2017, 4),
 (2017, 5),
 (2017, 6),
 (2017, 7),
 (2017, 8),
 (2017, 9),
 (2017, 10),
 (2017, 11),
 (2017, 12),
 (2018, 1),
 (2018, 2),
 (2018, 3),
 (2018, 4),
 (2018, 5),
 (2018, 6),
 (2018, 7),
 (2018, 8),
 (2018, 9),
 (2018, 10),
 (2018, 11),
 (2018, 12),
 (2019, 1),
 (2019, 2),
 (2019, 3),
 (2019, 4),
 (2019, 5),
 (2019, 6),
 (2019, 7),
 (2019, 8),
 (2019, 9),
 (2019, 10),
 (2019, 11),
 (2019, 12),
 (2020, 1),
 (2020, 2),
 (2020, 3),
 (2020, 4),
 (2020, 5),
 (2020, 6),
 (2020, 7),
 (2020, 8),
 (2020, 9),
 (2020, 10

In [37]:
to_rebuild_eu =  [ (int(x.split('.')[0][4:8]), int(x.split('.')[0][8:10])) for x in os.listdir('EU')]
to_rebuild_ind = [ (int(x.split('.')[0][4:8]), int(x.split('.')[0][8:10])) for x in os.listdir('INDIA')]
to_rebuild_usa = [ (int(x.split('.')[0][4:8]), int(x.split('.')[0][8:10])) for x in os.listdir('USA')]
to_rebuild_ch = [ (int(x.split('.')[0][4:8]), int(x.split('.')[0][8:10])) for x in os.listdir('CHINA/IMPORT')]
to_rebuild_uk = [ (int('20' + x.split('.')[0][6:8]), int(x.split('.')[0][8:10])) for x in os.listdir('UK/EXPORT')]
#to_rebuild_bel = [ (int(x.split('.')[0][4:8]), int(x.split('.')[0][8:10])) for x in os.listdir('BELARUS')]
to_rebuild_kz = [ (int(x.split('.')[0][4:8]), int(x.split('.')[0][8:10])) for x in os.listdir('KZ') if x[0] == 'd']
to_rebuild_jp = [ (int(x[4:8])) for x in os.listdir('JAPAN')]
to_rebuild_sk = [ (int(x.split('.')[0][4:8]), int(x.split('.')[0][8:10])) for x in os.listdir('SK') if x[0] == 'd']
to_rebuild_turkey = [ (int(x[4:8])) for x in os.listdir('TURKEY') if x[0] == 'd']
to_rebuild_kg = [ x[4:8] for x in os.listdir('KG') if x[9:12]=='csv']

In [51]:
if not os.path.exists('data_processed'):
    os.mkdir('data_processed')

In [29]:
refresh_ind(2025, '01')

Processing 2025-01


In [30]:
for i in (to_rebuild_ind):
    refresh_ind(i[0],str(i[1]).zfill(2))

Processing 2010-01
Processing 2010-02
Processing 2010-03
Processing 2010-04
Processing 2010-05
Processing 2010-06
Processing 2010-07
Processing 2010-08
Processing 2010-09
Processing 2010-10
Processing 2010-11
Processing 2010-12
Processing 2011-01
Processing 2011-02
Processing 2011-03
Processing 2011-04
Processing 2011-05
Processing 2011-06
Processing 2011-07
Processing 2011-08
Processing 2011-09
Processing 2011-10
Processing 2011-11
Processing 2011-12
Processing 2012-01
Processing 2012-02
Processing 2012-03
Processing 2012-04
Processing 2012-05
Processing 2012-06
Processing 2012-07
Processing 2012-08
Processing 2012-09
Processing 2012-10
Processing 2012-11
Processing 2012-12
Processing 2013-01
Processing 2013-02
Processing 2013-03
Processing 2013-04
Processing 2013-05
Processing 2013-06
Processing 2013-07
Processing 2013-08
Processing 2013-09
Processing 2013-10
Processing 2013-11
Processing 2013-12
Processing 2014-01
Processing 2014-02
Processing 2014-03
Processing 2014-04
Processing 2

In [17]:
for i in (to_rebuild_usa):
    refresh_usa(i[0],str(i[1]).zfill(2))

EmptyDataError: No columns to parse from file

In [22]:
to_rebuild_eu

[(1992, 1),
 (1992, 2),
 (1992, 3),
 (1992, 4),
 (1992, 5),
 (1992, 6),
 (1992, 7),
 (1992, 8),
 (1992, 9),
 (1992, 10),
 (1992, 11),
 (1992, 12),
 (1993, 1),
 (1993, 2),
 (1993, 3),
 (1993, 4),
 (1993, 5),
 (1993, 6),
 (1993, 7),
 (1993, 8),
 (1993, 9),
 (1993, 10),
 (1993, 11),
 (1993, 12),
 (1994, 1),
 (1994, 2),
 (1994, 3),
 (1994, 4),
 (1994, 5),
 (1994, 6),
 (1994, 7),
 (1994, 8),
 (1994, 9),
 (1994, 10),
 (1994, 11),
 (1994, 12),
 (1995, 1),
 (1995, 2),
 (1995, 3),
 (1995, 4),
 (1995, 5),
 (1995, 6),
 (1995, 7),
 (1995, 8),
 (1995, 9),
 (1995, 10),
 (1995, 11),
 (1995, 12),
 (1996, 1),
 (1996, 2),
 (1996, 3),
 (1996, 4),
 (1996, 5),
 (1996, 6),
 (1996, 7),
 (1996, 8),
 (1996, 9),
 (1996, 10),
 (1996, 11),
 (1996, 12),
 (1997, 1),
 (1997, 2),
 (1997, 3),
 (1997, 4),
 (1997, 5),
 (1997, 6),
 (1997, 7),
 (1997, 8),
 (1997, 9),
 (1997, 10),
 (1997, 11),
 (1997, 12),
 (1998, 1),
 (1998, 2),
 (1998, 3),
 (1998, 4),
 (1998, 5),
 (1998, 6),
 (1998, 7),
 (1998, 8),
 (1998, 9),
 (1998, 10

In [53]:
for i in tqdm(to_rebuild_eu):
    refresh_eu(i[0],str(i[1]).zfill(2))

  0%|          | 0/389 [00:00<?, ?it/s]

In [24]:
for i in tqdm(to_rebuild_ch):
    refresh_ch(i[0],str(i[1]).zfill(2), 'ЭК')
    refresh_ch(i[0],str(i[1]).zfill(2), 'ИМ')

  0%|          | 0/90 [00:00<?, ?it/s]

In [43]:
for i in tqdm(to_rebuild_uk):
    refresh_uk(i[0],str(i[1]).zfill(2), 'ЭК')
    refresh_uk(i[0],str(i[1]).zfill(2), 'ИМ')

  0%|          | 0/101 [00:00<?, ?it/s]

In [25]:
kurs_funt

,original_period,value
540,2002-01,0.707664
541,2002-02,0.705816
542,2002-03,0.702642
543,2002-04,0.686436
544,2002-05,0.681803
...,...,...
805,2024-02,0.790483
806,2024-03,0.792864
807,2024-04,0.797416
808,2024-05,0.786040


In [28]:
for i in (to_rebuild_kz):
    refresh_kz(i[0],str(i[1]).zfill(2))

C:\Users\m_salihov\AppData\Local\Temp\ipykernel_26316\4047982489.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_st['TNVED'] = df_st['TNVED'].fillna(method='ffill')
C:\Users\m_salihov\AppData\Local\Temp\ipykernel_26316\4047982489.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PERIOD'] = filename[7:11] + '-' + filename[11:13] + '-01'
C:\Users\m_salihov\AppData\Local\Temp\ipykernel_26316\4047982489.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_st['TNVED'] = df_st['TNVED'].fillna(method='ffill')
C:\Users\m_salihov\AppData\Local\Temp\ipykernel_26316\

ValueError: Unable to read workbook: could not assign names from KZ/data202203.xls.
This is most probably because the workbook source files contain some invalid XML.
Please see the exception for more details.

In [49]:
for i in range(2014, int(current_year)+1):
    refresh_jp(str(i))

In [26]:
for i in tqdm(to_rebuild_sk):
    refresh_sk(str(i[0]),str(i[1]).zfill(2))

  0%|          | 0/125 [00:00<?, ?it/s]

AttributeError: 'float' object has no attribute 'replace'

In [27]:
for i in to_rebuild_turkey:
    refresh_turkey(str(i))

2019
2020
2021


D:\Anaconda2022\envs\gie\lib\site-packages\ipykernel_launcher.py:2: DtypeWarning: Columns (0,1,2,3,5,12) have mixed types.Specify dtype option on import or set low_memory=False.
  


2022


D:\Anaconda2022\envs\gie\lib\site-packages\ipykernel_launcher.py:2: DtypeWarning: Columns (0,1,2,16) have mixed types.Specify dtype option on import or set low_memory=False.
  


2023


D:\Anaconda2022\envs\gie\lib\site-packages\ipykernel_launcher.py:2: DtypeWarning: Columns (0,1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  


2024


In [18]:
for i in tqdm(to_rebuild_kg):
    refresh_KG(i)

NameError: name 'to_rebuild_kg' is not defined

In [54]:
for i in countries:
    print(i)
    df = pd.read_csv(f'data_processed/{i}_full.csv', on_bad_lines='skip')
    file = 'data_processed/data.csv'
    if not os.path.exists(file):
        df.to_csv(file, header='column_names', index=False, encoding='utf-8-sig')
    else: # else it exists so append without writing the header
        df.to_csv(file, mode='a', header=False, index=False, encoding='utf-8-sig')

usa
eu
india
ch
uk
kz
jp
sk
kg


FileNotFoundError: [Errno 2] No such file or directory: 'data_processed/kg_full.csv'

In [100]:
df = pd.read_csv('data_processed/data.csv')

C:\Users\ilua\AppData\Local\Temp\ipykernel_15316\1493702622.py:1: DtypeWarning: Columns (3,4,6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data_processed/data.csv')


MemoryError: Unable to allocate 5.17 GiB for an array with shape (9, 77071998) and data type object

In [ ]:
df[df['EDIZM'].str.contains("filament", na=False)]